In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Data**

In [ ]:
X = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')
print("Train set:",X.shape)
print("Test set:",test.shape)

**Checking for empty cells**

In [ ]:
print("Missing data points in",X.isna().any().sum(),"columns of training set")
print("Missing data points in",test.isna().any().sum(),"columns of testing set")

**Data Visualization**

**handling dates**

In [ ]:
# Changes the datatype of date column from string to date 
def time_splitter(df,date):  
    df[date]         = pd.to_datetime(df[date])
    df['year']       = df[date].dt.year
    df['month']      = df[date].dt.month
    df['day']        = df[date].dt.day
    df['week']       = df[date].dt.isocalendar().week
    df['weekday']    = df[date].dt.weekday
    df['day_of_yr']  = df[date].dt.dayofyear
    df['quarter']    = df[date].dt.quarter
    return df

In [ ]:
X = time_splitter(X,'date')
test = time_splitter(test,'date')

In [ ]:
X.drop(['date'], axis=1, inplace=True)
test.drop(['date'], axis=1, inplace=True)
X.head()

**Handling Categorical Variables**

In [ ]:
#finding categorical variable
cate_var = [x for x in X.columns if X[x].dtype != 'int64']
cate_var

In [ ]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
dx= defaultdict(LabelEncoder)
 
x = X[cate_var]

x=x.apply(lambda x: dx[x.name].fit_transform(x))
X[cate_var]=x

In [ ]:
dt= defaultdict(LabelEncoder)
 
x = test[cate_var]

x=x.apply(lambda x: dt[x.name].fit_transform(x))
test[cate_var]=x

**splitting data**

In [ ]:
y = X.num_sold
X.drop(['num_sold'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(xtrain,ytrain)

In [ ]:
preds = model.predict(xvalid)

Importing Data

In [ ]:
from sklearn.metrics import mean_absolute_error
acc = mean_absolute_error(yvalid, preds)
acc

In [ ]:
oppreds = model.predict(test)

In [ ]:
output = pd.DataFrame({'row_id': test.row_id,
                       'num_sold': np.ceil(oppreds)})
output.to_csv('submission.csv', index=False)